# micrograd

In [1]:
import math
import numpy as np
import matplotlib as plt
%matplotlib inline

In [17]:
class Value:
    def __init__(self, data, _children=()):
        self.data = data
        self._prev = set(_children)
    
    def __repr__(self):
        return f"value: {self.data}"
    
    def __add__(self, other):
        return Value(self.data + other.data, (self, other))
    
    def __mul__(self, other):
        return Value(self.data * other.data, (self, other))

a = Value(2)
b = Value(-1)
print(a * b)

set()
value: -2
